In [1]:
#ray stop
#ray start --head --port=6379 --redis-password="cbgt2"
#ray start --address='10.162.14.77:6379' --redis-password='cbgt2'
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload 
#if it doesn't work, shut down the ray server and start it again


In [2]:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct_copy as popconstruct
import pdb

/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-08-05 16:15:15,425	INFO worker.py:727 -- Connecting to existing Ray cluster at address: 172.25.0.163:6379


In [51]:
pl = cbgt.Pipeline()

In [52]:
pl.celldefaults = pl[par.helper_cellparams]()
pl.popspecific = pl[par.helper_popspecific]()
pl.receptordefaults = pl[par.helper_receptor]()
pl.basestim = pl[par.helper_basestim]()
pl.dpmndefaults = pl[par.helper_dpmn]()
pl.d1defaults = pl[par.helper_d1]()
pl.d2defaults = pl[par.helper_d2]()
pl.actionchannels = pl[par.helper_actionchannels]()
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)
#pl.Q_support_params = pl[par.helper_init_Q_support_params]()

In [53]:
def codeblock_init_Q_support_params(self):
    self.Q_support_params = par.helper_init_Q_support_params()

def codeblock_init_Q_df(self):
    self.Q_df = par.helper_init_Q_df(self.channels)
    
def codeblock_update_chosen_action(self,action):
    self.Q_support_params = par.helper_update_chosen_action(self.Q_support_params,action)
    
def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = par.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults)
    


In [54]:
q_val_pipeline = cbgt.Pipeline()
q_val_pipeline.Q_support_params = q_val_pipeline[par.helper_init_Q_support_params]()
q_val_pipeline.Q_df = q_val_pipeline[par.helper_init_Q_df](pl.actionchannels)
q_val_pipeline.chosen_action = q_val_pipeline[par.helper_update_chosen_action](q_val_pipeline.Q_support_params,2)
q_val_pipeline.add(codeblock_update_Q_df)

pl.add(q_val_pipeline)



In [55]:
environment = {
    'params': None, 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'newpathways' : None,
    'Q_support_params': None,
    'Q_df':None,
    
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

(pid=3680107) update chosen action
(pid=3680107) update chosen action
(pid=3680107) In update_Q_df
(pid=3680107) Q_support_params
(pid=3680107)    bayes_unif_min  bayes_unif_max  ...  reward_value  chosen_action
(pid=3680107) 0             0.0             2.0  ...            -1              2
(pid=3680107) 
(pid=3680107) [1 rows x 8 columns]
(pid=3680107) Q_df
(pid=3680107)    Q_val  action
(pid=3680107) 0  0.500       1
(pid=3680107) 1 -0.175       2
(pid=3680107) In update_Q_df
(pid=3680107) Q_support_params
(pid=3680107)    bayes_unif_min  bayes_unif_max  ...  reward_value  chosen_action
(pid=3680107) 0             0.0             2.0  ...            -1              2
(pid=3680107) 
(pid=3680107) [1 rows x 8 columns]
(pid=3680107) Q_df
(pid=3680107)    Q_val  action
(pid=3680107) 0  0.500       1
(pid=3680107) 1 -0.175       2


In [56]:
print(results)

{'params': None, 'pops': None, 'receps': None, 'base': None, 'dpmns': None, 'd1': None, 'd2': None, 'channels': None, 'newpathways': None, 'Q_support_params':    bayes_unif_min  bayes_unif_max  bayes_H  bayes_sF  q_alpha  dpmn_CPP_scale  \
0             0.0             2.0     0.05      1.25     0.45            15.0   

   reward_value  chosen_action  
0            -1              2  , 'Q_df':    Q_val  action
0  0.500       1
1 -0.175       2, 'celldefaults':      N     C Taum RestPot ResetPot Threshold RestPot_ca Alpha_ca Tau_ca  \
0  75`  0.5`  20`    -70`     -55`      -50`       -85`     0.5`    80`   

  Eff_ca  ... g_adr_max Vadr_h Vadr_s ADRRevPot g_k_max  Vk_h  Vk_s tau_k_max  \
0   0.0`  ...        0`  -100`    10`      -90`      0`  -34`  6.5`        8`   

  n_k   h  
0  0`  1`  

[1 rows x 25 columns], 'popspecific': {'LIP': {'N': [204]}, 'FSI': {'C': [0.2], 'Taum': [10]}, 'GPeP': {'N': [750], 'g_T': [0.06], 'Taum': [20]}, 'STNE': {'N': [750], 'g_T': [0.06]}, 'LIPI': {'N':

0   -1.5
Name: reward_value, dtype: float64


ValueError: assignment destination is read-only